# 豆瓣爬虫分步骤演示

## 使用Python的requests库读取网页并提取返回的数据

下面演示如何获取到https://movie.douban.com/subject/11026735/ 的HTML代码

Tips：如何快速获取到对于的python代码？

![拷贝浏览器请求](resources/douban-1.png)
![转换成代码](resources/douban-2.png)

In [2]:
import requests

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://movie.douban.com/subject/11026735/comments?start=200&limit=20&status=P&sort=new_score',
    'Accept-Language': 'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
}

response = requests.get('https://movie.douban.com/subject/11026735/', headers=headers)

显示前几百个字符

In [3]:
response.text[0:500]

'<!DOCTYPE html>\n<html lang="zh-CN" class="ua-mac ua-webkit">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <meta name="referrer" content="always">\n    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />\n    <title>\n        超能陆战队 (豆瓣)\n</title>\n    \n    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />\n    <meta http-equiv="Pragma" content="no-cache">\n    <me'

## 通过正则表达式获取电影标题名称

正则表达式测试:https://regex101.com/r/AFRdGf/2

In [4]:
import re

regex = r"\"name\":\s\"(?P<name>.*?)\",\s*\"url\":\s\"(?P<url>.*?)\",\s*\"image\":\s\"(?P<image>.*?)\""
re.findall(regex,response.text)

[('超能陆战队 Big Hero 6',
  '/subject/11026735/',
  'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p2614500883.webp')]

## 对于豆瓣，可以提取json+ld来获取相关信息

相关正则表达式：https://regex101.com/r/Ac3L5n/1

In [5]:
import json

regex = r"ld\+json\">(.*?)</script>"
jsonld = re.findall(regex,response.text, re.MULTILINE | re.DOTALL)[0]

movie = json.loads(jsonld)
movie['name']

'超能陆战队 Big Hero 6'

In [6]:
# 编剧
[x['name'] for x in movie['author']]

['乔丹·罗伯茨 Jordan Roberts',
 '丹尼尔·吉尔森 Daniel Gerson',
 '罗伯特·L·贝尔德 Robert L. Baird',
 '唐·霍尔 Don Hall',
 '邓肯·鲁洛 Duncan Rouleau',
 '史提芬·T·西格尔 Steven T. Seagle',
 '乔·凯西 Joe Casey',
 '乔·凯利 Joe Kelly']

In [7]:
# 演员
[x['name'] for x in movie['actor']]

['斯科特·安第斯 Scott Adsit',
 '瑞恩·波特 Ryan Potter',
 '丹尼尔·海尼 Daniel Henney',
 'T·J·米勒 T.J. Miller',
 '杰米·钟 Jamie Chung',
 '小达蒙·韦恩斯 Damon Wayans Jr.',
 '珍尼希斯·罗德里格兹 Genesis Rodriguez',
 '詹姆斯·克伦威尔 James Cromwell',
 '艾伦·图代克 Alan Tudyk',
 '玛娅·鲁道夫 Maya Rudolph',
 '亚布拉哈姆·本鲁比 Abraham Benrubi',
 '凯蒂·洛斯 Katie Lowes',
 '比利·布什 Billy Bush',
 '丹尼尔·吉尔森 Daniel Gerson',
 '保罗·布里格斯 Paul Briggs',
 '夏洛特·古列齐 Charlotte Gulezian',
 'Dan Howell',
 '乔西·特立尼达 Josie Trinidad',
 '菅野美穗 Miho Kanno',
 '斯坦·李 Stan Lee',
 '大卫·肖内西 David Shaughnessy',
 '查尔斯·阿德勒 Charles Adler',
 '郝祥海 Xianghai Hao']

In [8]:
# 评分
float(movie['aggregateRating']['ratingValue'])

8.7

## 通过BeautifulSoup提取

BeautifulSoup提供了解析XML、HTML的各种方法，并且提供了选择器可以更快的定位到需要的元素。
CSS selector可以通过浏览器直接得到

In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')
[x.text for x in soup.select("#info > span.actor > span.attrs > a")]

['斯科特·安第斯',
 '瑞恩·波特',
 '丹尼尔·海尼',
 'T·J·米勒',
 '杰米·钟',
 '小达蒙·韦恩斯',
 '珍尼希斯·罗德里格兹',
 '詹姆斯·克伦威尔',
 '艾伦·图代克',
 '玛娅·鲁道夫',
 '亚布拉哈姆·本鲁比',
 '凯蒂·洛斯',
 '比利·布什',
 '丹尼尔·吉尔森',
 '保罗·布里格斯']

# 小练习
取消下面代码的注释并修改下面的代码，使之能够读取到该电影的影评信息：https://movie.douban.com/subject/11026735/comments

In [10]:
# url = "填入正确的url"
# response = requests.get(url, headers=headers)
# response.content[0:500]

Tips: 获取评论 Copy selector得到CSS表达式
`#comments > div:nth-child(1) > div.comment > p > span`
注意这里有一个`div:nth-child(1)`，指的是第一个div元素，如果要遍历所有的，需要删除`:nth-child(1)`

![](./resources/douban-practice-tip-1.png)


In [11]:
soup = BeautifulSoup(response.content, 'html.parser')
[x.text for x in soup.select("填入正确的css selector")]

[]

In [12]:
[x.text for x in soup.select("#comments > div > div.comment > h3 > span.comment-info > a")]

[]

找到下一页的链接并进行遍历

In [13]:
baseurl = "https://movie.douban.com/subject/11026735/comments"
nextpage_url = ""
while True:
    response = requests.get(baseurl + nextpage_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    nextpage = soup.select("#paginator > a.next")
    content = [x.text for x in soup.select("#comments > div > div.comment > p > span")]
    print("这一页所有的评论", content)
    
    if len(nextpage) == 0:
        break
        
    nextpage_url = nextpage[0]['href']
    print("正在获取", nextpage_url)
    if nextpage_url is None:
        break

这一页所有的评论 ["迪士尼最新的Hyperion渲染器，渲染旧金山全景，几千个光源，无数的precedural shading...碉堡... 剧情的话Hero's journy，感情带动很好。动画感觉比Dragon2有看头~ 忘了说一句，记得看彩蛋~ Stan Lee的哏儿~", '三番和东京，动画和机器人，樱花和Caltech，我们的城市我们的大学我们的专业，这部片子对我的意义不一样，我真的惊。呆。了。对最近在想皮肤材质和场景建模和粒子特效的我也格外有教科书的意义。再见Monster Co，你现在排第二了！（DC在这个我所看过的最彩的彩蛋里被Marvel黑出翔~~~！！！！', '史上最萌胖子诞生！要抱抱！彩蛋里的斯坦·李也是蛮拼的！', '故事不新鲜，但角色有新意。几处萌点和泪点全部来自大白，大白的陪伴，让人想起的不仅仅是：大熊身边的哆啦A梦、教授帕克的忠犬八公、坚守地球的WALL·E、邻家的多多龙、维克多的科学怪狗、小嗝嗝身边的无牙、库珀身边的机器人Case、山姆身边的Gerty、Wallace身边的Gromit...', '又萌又燃有内涵，给大白五星满分！本片美指晚餐可以加个鸡腿，不过私心以为赛博和风的城市设计应该找帝国少年来做的。看完也想在飘满涡轮驱动风船的城市里找个呆萌呆萌的大白，一起坐在鱼风船上看夕阳……等等，大白和hiro酱好像都是男的？#不腐不是迪士尼#', '好想戳爆胖子看会发生什么……', '集合全世界的萌于一身的大白，让机器人的暖心地位达到了新高度，超级英雄携手人工智能更集合了时下最热的观感体验：飙车、飞行与触手的魔性侵入。当然最棒的还是日本与旧金山的融合，做梦都希望它是真的存在~彩蛋某老爷子又怒刷存在感。最后我想喊一万句，大白嫁我！五星全给大白！细节要戳死了！', 'balalala', '身体像男生般强壮，心思像女生般细腻，平时像孩子般呆萌，遇事像家长般护娃。一举赢尽所有主流观影群~', '为大白(●––●)打五星，太萌了，挺好看的。', '大胖胖终于将瓦力拱下了机器人萌货的头名宝座，日漫元素和迪斯尼风格合体开出了一朵大奇葩！故事其实就是“蜘蛛侠”1+2，但想象力和细节太挠人心窝子了…', '以后打听动画，不用再问好吗，只需问人萌吗？动画好不好，全看萌不萌，动画片彻底进入萌时代。', '又是一个故事平平只靠细节讨巧的动画，但是没办法，